# 02_NIFTI_to_25DJPG.ipynb

This script generate the 2.5D images (in .jpeg format) from the 3D NIFTI (.nii) scans. 

The binary labels (abnormal vs. normal) are also generated for the 2.5D images. The labels of the 2.5D images (saved in `local_label.csv`) follow the label of their associated NIFTI scan.

Required files:
- the 3D CT scans in NIFTI (.nii) format
- `label.csv`

Generate:
- the 2.5D images in JPEG (.jpeg) format
- `local_label.csv`

In [ ]:
import sys
repo_dir = <PATH TO THIS REPO> # local path to this repo.
sys.path.insert(0, repo_dir)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib
from scipy import ndimage
from PIL import Image
import os
from tqdm import tqdm

# settings to display all columns
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 150)

# utilitiy files located in the src folder
from src.util_image import read_nifti_file, normalise, 
from src.util_image import resize_imagesize, get_volume_from_nifti

In [ ]:
# locating the folder paths used in this script.
work_dir = <PATH TO project_dir> # path to the directory of the current project (project master path)
data_folder_path = work_dir + "CT_data/" # where the data is stored
image_folder_path = data_folder_path + "CT_NIFTI_cropped_combineLR/" # directory of the NIFTI files that generated by 01_Crop_NIFTI_all.ipynb 
targetpath = data_folder_path + "CT_25DJPG_cropped_combineLR_downsampled/" # directory to save the generated 2.5D JPEG

# load the binary label and add the full file path of the NIFTI files
df_label = pd.read_csv(data_folder_path + "/label.csv")
df_label["filepath"] = image_folder_path
df_label["filepath"] = df_label["filepath"].str.cat(df_label["image_name"],sep="")
display(df_label)

## Convert NIFTI to 2.5D JPEG
All generated 2.5D images saved in: <`targetpath`>. Binary labels of the 2.5D images saved in: `<targetpath>/local_label.csv`.

In [5]:
# set the target image size of the 2.5D images. Images with higher resolution will be downsampled to this size.
target_imagesize = [240,120,3]

In [ ]:
scan_list = os.listdir(image_folder_path)
df_local_label=pd.DataFrame()
for path in tqdm(df_label.filepath):
    series_name = os.path.basename(path)
    if series_name in scan_list:
        # get the pixel data from the NIFTI file
        volume = get_volume_from_nifti(path)
        image_depth = volume.shape[2]
        nifti_imagename = os.path.splitext(series_name)[0]
        for slide_i in range(image_depth-2):
            # construct the 2.5D images
            volume_3channel = volume[:,:,slide_i:slide_i+3]
            volume_3channel = resize_imagesize(volume_3channel,target_imagesize)
            # save the constructed 2.5D image
            target_file_name = nifti_imagename+"_"+str(slide_i)+".jpeg"
            target_file_path = targetpath + target_file_name
            plt.imsave(target_file_path,volume_3channel)
            # record the associated binary label of the 2.5D image
            df_local = df_label[df_label["filepath"]==path][["PatientName","image_name","abnormal"]]
            df_local["jpeg_name"] = target_file_name
            df_local["image_size"] = [volume_3channel.shape]
            df_local_label = df_local_label.append(df_local, ignore_index=True)

# save all binary labels of the generated 2.5D images
df_local_label.to_csv(targetpath + "local_label.csv",index=False)

Example format of the `local_label.csv`:

|   | PatientName |   image_name | abnormal |       jpeg_name |    image_size |
|--:|------------:|-------------:|---------:|----------------:|--------------:|
| 0 |   Patient_1 |      XXX.nii |        1 |      XXX_0.jpeg | (240, 120, 3) |
| 1 |   Patient_1 |      XXX.nii |        1 |      XXX_1.jpeg | (240, 120, 3) |
|   |         ... |          ... |      ... |             ... |           ... |

where:
- `PatientName`: the anonymous name of the Patient.
- `image_name`: the name of the NIFTI file.
- `abnormal`: the binary label (1: abnormal; 0: normal), same as the label for the NIFTI file.
- `jpeg_name`: the name of the 2.5D JPEG file. In the format of `XXX_n.jpeg`, where `XXX` is the name of the NIFTI file it generated from, `n` is the number order of the 2.5D images generated from the NIFTI file (number count starting from `0`).
- `image_size`: record of the image size, all have `(240, 120, 3)` in this case.